# BlueField DPUs
BlueField is a next-generation Data Processing Unit (DPU) developed by NVIDIA (formerly Mellanox Technologies). It integrates powerful computing, networking, and storage acceleration capabilities into a single device. Designed for modern data centers, BlueField DPUs offer hardware-accelerated data processing, efficient resource management, and enhanced security features.

## Key Features of BlueField-3
- Integrated Compute and Networking:
  - Combines an ARM-based SoC (System-on-Chip) with a ConnectX-7 network interface controller (NIC).
  - Offers hardware acceleration for networking and storage operations.

- Advanced Networking:
  - Supports up to 400Gb/s Ethernet or InfiniBand networking.
  - Equipped with SR-IOV, RDMA, and GPUDirect Storage capabilities.
    
- Storage Acceleration:
  - Enables offloading for NVMe over Fabrics (NVMe-oF).
  - Provides support for RAID and data mirroring.

- Security Capabilities:
  - Includes a hardware root of trust and secure boot features.
  - Offers real-time encryption, data isolation, and zero-trust security models.

## Create a slice using BlueField-3 SmartNICs 

This notebook shows how to create an L3 network using BlueField-3 SmartNICs and connect compute nodes to it using FABlib's automatic configuration functionality.

## Import the FABlib Library


In [ ]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network
import ipaddress

from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config();

## Create the Experiment Slice

This example sets up two nodes, each equipped with a BlueField-3 NIC, connected to an L3 network. 

Each node is created with a single NIC component, utilizing the `NIC_ConnectX_7_400` and `NIC_Basic` models. These components are attached to the node via PCI passthrough. A list of other available NIC models is provided below. To retrieve the interfaces associated with a NIC component, use the `get_interfaces()` method. Many dedicated NICs feature multiple ports, either of which can be connected to the network.

The node connected to the BlueField SmartNIC runs the `dpu_ubuntu_24` image, which includes DOCA version `2.9.1` by default. Alternatively, users can deploy VMs with the `default_ubuntu_24` image and manually install a different DOCA version as needed.

For automatic configuration, specify a subnet for the network and set the interface mode to `auto` using `iface1.set_mode('auto')` before submitting the request. With this setup, FABlib assigns an IP address from the subnet and configures the device during post-boot setup. Additionally, routes can be pre-configured before submitting the request.

### Available BlueField NIC Component Models:
- **NIC_ConnectX_7_100**: 100 Gbps Dedicated Mellanox BlueField-3 ConnectX-7 PCI Device (2 Ports)
- **NIC_ConnectX_7_400**: 400 Gbps Dedicated Mellanox BlueField-3 ConnectX-7 PCI Device (2 Ports)

In [ ]:
slice_name = 'MySlice-bluefields'
#site = fablib.get_random_site()
site="SALT"
print(f"Site: {site}")

node1_name = 'Node1'
node2_name = 'Node2'

network_name='net1'

node1_image = "dpu_ubuntu_24"
node2_image = "default_ubuntu_24"

In [ ]:
#Create Slice
slice = fablib.new_slice(name=slice_name)

# Network
#net1 = slice.add_l2network(name=network_name, subnet=IPv4Network("192.168.1.0/24"))
net1 = slice.add_l3network(name=network_name)

# Node1
node1 = slice.add_node(name=node1_name, site=site, image=node1_image)
dpu = node1.add_component(model='NIC_ConnectX_7_400', name='nic1')
iface1 =  dpu.get_interfaces()[0]
iface1.set_mode('auto')
net1.add_interface(iface1)

iface2 =  dpu.get_interfaces()[1]
iface2.set_mode('auto')
net1.add_interface(iface2)

# Node2
node2 = slice.add_node(name=node2_name, site=site, image=node2_image)
iface3 = node2.add_component(model='NIC_Basic', name='nic1').get_interfaces()[0]
iface3.set_mode('auto')
net1.add_interface(iface3)


#Submit Slice Request
slice.submit();

## Configure the Bluefield Smart NIC

The BlueField SmartNIC is configured by assigning the private IP address `192.168.100.1` to the `tmfifo_net0` device, enabling communication and management of the BlueField DPU. Additionally, the BlueField bundle (BFB) is installed on the DPU via the designated RShim interface, ensuring firmware updates and configuration optimizations for enhanced data center performance.

When using the `dpu_ubuntu_24` image for a node (e.g., `Node1`) connected to the BlueField SmartNIC, the BFB image is available by default at:  
`/opt/bf-bundle/bf-bundle-2.9.1-40_24.11_ubuntu-24.04_prod.bfb`.  
The installation process is initiated when `bluefield.configure()` is executed.

To run custom commands, provide a list of command strings as an argument to `bluefield.configure(commands)`.

In [ ]:
slice = fablib.get_slice(slice_name)

node1 = slice.get_node(name=node1_name) 
bluefield = node1.get_component(name='nic1')
output = bluefield.configure()

### Post-Configuration: Reboot and Set Up DPU Management Interface

Reapply the network configuration on the node connected to the BlueField. This step is only effective when automatic configuration is in use, i.e., when interfaces are set with `iface1.set_mode('auto')` or `iface1.set_mode('config')`.

After applying the configuration, reboot the VM connected to the DPU, since the VM OS may not always detect the interfaces automatically. The following cells perform these steps:

1. **Reboot the VM** and wait for SSH to become available again.
2. **Assign the management IP** `192.168.100.1/24` to the `tmfifo_net0` interface, which is used to communicate with the DPU over the RShim channel.
3. **Bring up the `tmfifo_net0` interface** to enable connectivity to the DPU.
4. **Generate an SSH key** on the host VM, which will be used to set up passwordless SSH access to the DPU.

In [ ]:
node1.execute("sudo reboot")
slice.wait_ssh()

In [ ]:
stdout, stderr = node1.execute("sudo ip addr add 192.168.100.1/24 dev tmfifo_net0")

In [ ]:
stdout, stderr = node1.execute("sudo ip link set tmfifo_net0 up")

In [ ]:
stdout, stderr = node1.execute("ssh-keygen -t ed25519 -f ~/.ssh/id_ed25519 -N ''")

## Accessing the DPU (Manual Steps)

The following steps must be performed manually from a terminal session on the host VM.

#### SSH Access
SSH into the DPU from the host VM:
```
ssh ubuntu@192.168.100.2
```
After the BFB image is pushed to the DPU, the default credentials are:  
- **Username:** `ubuntu`  
- **Password:** `ubuntu` (you will be prompted to change it on first login)

**Note:** Remember the password you set.

Next, copy your SSH key to the DPU to enable passwordless access:
```
ssh-copy-id ubuntu@192.168.100.2
```
This will prompt for the password you just set. Once complete, you can SSH to the DPU without entering a password.

#### Console Access
If SSH access is unavailable, you can connect to the DPU via the console interface:
```
screen /dev/rshim0/console
```
This provides an alternative way to manage the SmartNIC if SSH connectivity is lost.

## Enable Internet Access on DPU

The following cells configure NAT-based internet access on the DPU through the host VM's management interface. The steps are:

1. **Upload the node tools** scripts to the host VM.
2. **Detect the management IP version** (IPv4 or IPv6) of the host VM.
3. **Run the `bf3_rshim.sh` script** with the appropriate mode to set up NAT forwarding and routing so the DPU can reach the internet via the host.
4. **Re-apply the network configuration** on the host VM interfaces and list all configured interfaces.

In [ ]:
node1.upload_directory("node_tools", ".")

In [ ]:
import ipaddress
ip = ipaddress.ip_address(node1.get_management_ip())

In [ ]:
if ip.version == 4:
    stdout, stderr = node1.execute("sudo ./node_tools/bf3_rshim.sh --mode ipv4")
else:
    stdout, stderr = node1.execute("sudo ./node_tools/bf3_rshim.sh --mode ipv6")

### Re-apply the network config post re-imaging

Reapply the network configuration on the node connected to the BlueField. This step is only effective when automatic configuration is in use, i.e., when interfaces are set with `iface1.set_mode('auto')` or `iface1.set_mode('config')`.

After applying the configuration, reboot the VM connected to the DPU, since the VM OS may not always detect the interfaces automatically.


In [ ]:
node1.config()
slice.list_interfaces();

In [ ]:
for iface in node1.get_interfaces():
    print(f"Setting IP on {iface.get_name()}")
    stdout, stderr = node1.execute(f"sudo ifconfig {iface.get_physical_os_interface_name()} up")
    stdout, stderr = node1.execute(f"sudo ip addr add {iface.get_ip_addr()}/24 dev {iface.get_physical_os_interface_name()}")

In [ ]:
for n in slice.get_nodes():
    print(f"Listing IPs on {n.get_name()}")
    stdout, stderr = n.execute("ip addr")
    print("===============================================================================================================")
    print()

## Run the Experiment

With automatic configuration the slice is ready for experimentation after it becomes active.  Note that automatic configuration works well when saving slices to a file and reinstantiating the slice.  Configuration tasks can be stored in the saved slice, reducing the complexity of notebooks and other runtime steps.

We will find the ping round trip time for this pair of sites.  Your experiment should be more interesting!

**Note:** If the ping fails, try rerunning the cell above to ensure the IP address is properly configured, then attempt the ping again.

In [ ]:
slice = fablib.get_slice(slice_name)

node1 = slice.get_node(name=node1_name)        
node2 = slice.get_node(name=node2_name)           

node2_addr = node2.get_interface(network_name=network_name).get_ip_addr()

stdout, stderr = node1.execute(f'ping -c 5 {node2_addr}')

## Delete the Slice

Please delete your slice when you are done with your experiment.

In [ ]:
slice = fablib.get_slice(slice_name)
slice.delete()